[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmalbornoz/SimpleRAG/blob/main/1_simple_rag_example.ipynb)

# Simple RAG Example #1
## Dr José M Albornoz
### April 2024

In this notebook we will illustrate a basic RAG workflow using as context a fragement of E.A. Poe's "The Cask of Amontillado"

In [1]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/RAG/')

Mounted at /content/drive


In [2]:
! pip install openai langchain unstructured pdf2image tabulate libmagic num2words tiktoken ctransformers --no-warn-script-location > /dev/null

# CHROMADB SENTENCE TRANSFORMERS

In [3]:
#!pip install faiss-cpu
!pip install sentence-transformers --no-warn-script-location > /dev/null
!pip install chromadb --no-warn-script-location > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [4]:
! pip freeze | grep langchain

langchain==0.1.17
langchain-community==0.0.36
langchain-core==0.1.50
langchain-text-splitters==0.0.1


In [5]:
import chromadb

client = chromadb.Client()
#client.delete_collection("rag_demo")
collection = client.get_or_create_collection("rag_demo")

collection.add(
    documents=[""" "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess. """, "This is a document which is dummy"],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["What did the narrator do to his friend?"],
    n_results=1
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 41.4MiB/s]


In [6]:
from pprint import pprint
context_extracted = results['documents'][0][0]
pprint(context_extracted)

(' "He is an ignoramus," interrupted my friend, as he stepped unsteadily\n'
 'forward, while I followed immediately at his heels.  In an instant he\n'
 'had reached the extremity of the niche, and finding his progress\n'
 'arrested by the rock, stood stupidly bewildered.  A moment more and I\n'
 'had fettered him to the granite.  In its surface were two iron staples,\n'
 'distant from each other about two feet, horizontally.  From one of\n'
 'these depended a short chain, from the other a padlock.  Throwing the\n'
 'links about his waist, it was but the work of a few seconds to secure\n'
 'it.  He was too much astounded to resist.  Withdrawing the key I\n'
 'stepped back from the recess. ')


 # BASIC RAG WORKFLOW

## Llama-2-7B-Chat-GGML

In [7]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 20, 'temperature': 0}
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML', config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [8]:
from langchain import PromptTemplate, LLMChain
template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
<</SYS>>

Answer the question below from context below :
{context}
{question} [/INST]
"""

question_p = """What did the narrator do to his friend?"""
context_p = """ "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess."""

prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The narrator secured the friend with a chain and padlock.'

## Falcon-7b-instruct

In [9]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 20, 'temperature': 0}
llm = CTransformers(model='TheBloke/falcon-7b-instruct-GGML', config=config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

falcon-7b-instruct.ggccv1.q4_0.bin:   0%|          | 0.00/4.06G [00:00<?, ?B/s]

In [10]:
from langchain import PromptTemplate, LLMChain
template = """
User: {context}
User: {question}
Assistant:
"""

question_p = """What did the narrator do to his friend?"""
context_p = """ "He is an ignoramus," interrupted my friend, as he stepped unsteadily
forward, while I followed immediately at his heels.  In an instant he
had reached the extremity of the niche, and finding his progress
arrested by the rock, stood stupidly bewildered.  A moment more and I
had fettered him to the granite.  In its surface were two iron staples,
distant from each other about two feet, horizontally.  From one of
these depended a short chain, from the other a padlock.  Throwing the
links about his waist, it was but the work of a few seconds to secure
it.  He was too much astounded to resist.  Withdrawing the key I
stepped back from the recess."""

prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

'The narrator locked his friend in a cell.\nUser '